In [5]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3

Define fields except firm name

In [6]:
fields = ["Registered_Address", "CEO", "Establishment_Year", "Number_Of_Employees", "Revenue_Size" ,
        "Website", "NAICS_Code", "SIC_Code", "Status" ]

Model Config

In [7]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-8b",
  generation_config= {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
    }
 )

chat_session = model.start_chat(
  history=[
  ]
)

In [8]:
response = chat_session.send_message("Hello there")
print(response.text)

Hello! How can I help you today?



In [9]:
def form_prompt(context, query, data):
    prompt = f"""
    Context:
    {context}

    Query:
    {query}

    Relevant Data:
    {data}
    """

    return prompt

In [4]:
general_context = """
You will be assisting me with filling in data fields for a firm database I am building.
I will tell you the name of the firm i am interested in, and the field I want you to fill.
I will give you potentially relevant information I have gained from web search results and the scraped contents of certain websites that I gathered by searching for the firm name and field and the field. 
The text will be in Json format.
You will give your answer by simply stating the value of the field I am interested in. 
Do not form sentences, just give the value of the field.
If you have absolutely no idea about the answer, then answer with 'null' .
"""

### Open firm database

In [3]:
use_stored_web_search_data = True

In [1]:
# Table to store the results
conn = sqlite3.connect("firm_database_llm.db")
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS firm_properties (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               Firm_Name TEXT NOT NULL,
               Registered_Address TEXT,
               CEO TEXT,
               Establishment_Year INT,
               Number_Of_Employees INT,
               Revenue_Size INT,
               Website TEXT,
               NAICS_Code INT,
               SIC_Code INT,
               Status TEXT
               )
               ''')

conn_web_results = sqlite3.connect("firms_search_results.db")
cursor_web_results = conn_web_results.cursor()

NameError: name 'sqlite3' is not defined

Loop through firm names in "firm_search_results.db"

In [ ]:
cursor_web_results.execute(''' SELECT id, Firm_Name FROM firms_search_results ''')
firm_web_search_results = cursor_web_results.fetchall()

for web_search_result in firm_web_search_results:
    firm_id = web_search_result[0]
    firm_name = web_search_result[1]
    # TODO check if firm is already in the firm_properties table

    for field in fields:
    # TODO check if field is filled in in the firm_properties table
        if use_stored_web_search_data:
            cursor_web_results.execute(f''' SELECT {field} FROM firms_search_results WHERE id = ? AND Firm_Name = ?'''(firm_id,firm_name))
            web_search_result = cursor_web_results.fetchone()
            web_search_result = json.loads(web_search_result[1]) # check indexing here
        else:
            pass


    prompt = form_prompt(general_context, f"Fill in the field {field} for the firm {firm_name}", web_search_result[field])

    success = False
    failure_counter = 0
    while not success and failure_counter < 3:
        try:
            response = chat_session.send_message(prompt).text
            # print("Success for ", firm_name, field)
            success = True
        except Exception as e:
            print("Exception occured: ", e)
            time.sleep(1) 
            failure_counter += 1
            
    # Update database
    cursor.execute(f''' INSERT INTO firm_properties (Firm_Name, {field}) VALUES ('{firm_name}', '{response.text}') ''')
    conn.commit()

In [14]:
conn.close()
cursor.close()

ProgrammingError: Cannot operate on a closed database.